In [1]:
import diffusers
from diffusers import UNet2DConditionModel
from PIL import Image
import sys
from transformers import CLIPVisionModel, CLIPImageProcessor
import torch

sys.path.append("/home/aihao/workspace/StableDiffusionReferenceOnly/src")
from stable_diffusion_reference_only.pipelines.stable_diffusion_reference_only_pipeline import (
    StableDiffusionReferenceOnlyPipeline,
)
from stable_diffusion_reference_only.models.unet_2d_dobule_condition import (
    UNet2DDobuleConditionModel,
)
from diffusers.schedulers import DDPMScheduler
import json

In [2]:
unet_config_path = "/home/aihao/workspace/StableDiffusionReferenceOnly/src/stable_diffusion_reference_only/models/unet-2-1.json"
pretrained_unet_path = "stabilityai/stable-diffusion-2-1"
pretrained_image_encoder_path = "openai/clip-vit-large-patch14"

# unet_config_path = "/home/aihao/workspace/StableDiffusionReferenceOnly/src/stable_diffusion_reference_only/models/unet_xl-base-1.0.json"
# pretrained_unet_path = "stabilityai/stable-diffusion-xl-base-1.0"
# pretrained_image_encoder_path = "laion/CLIP-ViT-bigG-14-laion2B-39B-b160k"

In [3]:
with open(unet_config_path) as f:
    unet_config = json.load(f)

In [4]:
unet_config

{'_class_name': 'UNet2DDobuleConditionModel',
 '_diffusers_version': '0.10.0.dev0',
 'act_fn': 'silu',
 'attention_head_dim': [5, 10, 20, 20],
 'block_out_channels': [320, 640, 1280, 1280],
 'center_input_sample': False,
 'cross_attention_dim': 1024,
 'down_block_types': ['CrossAttnDownBlock2D',
  'CrossAttnDownBlock2D',
  'CrossAttnDownBlock2D',
  'DownBlock2D'],
 'downsample_padding': 1,
 'dual_cross_attention': False,
 'flip_sin_to_cos': True,
 'freq_shift': 0,
 'in_channels': 4,
 'layers_per_block': 2,
 'mid_block_scale_factor': 1,
 'norm_eps': 1e-05,
 'norm_num_groups': 32,
 'num_class_embeds': None,
 'only_cross_attention': False,
 'out_channels': 4,
 'sample_size': 96,
 'up_block_types': ['UpBlock2D',
  'CrossAttnUpBlock2D',
  'CrossAttnUpBlock2D',
  'CrossAttnUpBlock2D'],
 'use_linear_projection': True,
 'upcast_attention': True}

In [5]:
unet=UNet2DDobuleConditionModel.from_config(unet_config)

In [6]:
pretrained_unet = UNet2DConditionModel.from_pretrained(
    pretrained_unet_path, subfolder="unet"
)

In [8]:
unet_parameters = unet.state_dict()
pretrained_unet_parameters = pretrained_unet.state_dict()
for key in unet_parameters:
    if key in pretrained_unet_parameters:
        if unet_parameters[key].shape == pretrained_unet_parameters[key].shape:
            unet_parameters[key] = pretrained_unet_parameters[key]
        elif unet_parameters[key].shape < pretrained_unet_parameters[key].shape:
            print(key)
            unet_parameters[key] = pretrained_unet_parameters[key][
                0 : unet_parameters[key].shape[0],
                0 : unet_parameters[key].shape[1],
            ]
        else:
            print(key)
            unet_parameters[key] = torch.nn.functional.pad(
                pretrained_unet_parameters[key],
                (
                    0,
                    unet_parameters[key].shape[1]
                    - pretrained_unet_parameters[key].shape[1],
                    0,
                    0,
                ),
            )

In [9]:
unet.load_state_dict(unet_parameters)

<All keys matched successfully>

In [10]:
for key in unet.state_dict():
    if key in pretrained_unet.state_dict():
        print(unet.state_dict()[key] == pretrained_unet.state_dict()[key])

tensor([[[[True, True, True],
          [True, True, True],
          [True, True, True]],

         [[True, True, True],
          [True, True, True],
          [True, True, True]],

         [[True, True, True],
          [True, True, True],
          [True, True, True]],

         [[True, True, True],
          [True, True, True],
          [True, True, True]]],


        [[[True, True, True],
          [True, True, True],
          [True, True, True]],

         [[True, True, True],
          [True, True, True],
          [True, True, True]],

         [[True, True, True],
          [True, True, True],
          [True, True, True]],

         [[True, True, True],
          [True, True, True],
          [True, True, True]]],


        [[[True, True, True],
          [True, True, True],
          [True, True, True]],

         [[True, True, True],
          [True, True, True],
          [True, True, True]],

         [[True, True, True],
          [True, True, True],
          [True,

In [11]:
# vae = diffusers.AutoencoderKL.from_pretrained("stabilityai/sdxl-vae")
vae = diffusers.AutoencoderKL.from_pretrained(
    "stabilityai/stable-diffusion-2-1", subfolder="vae"
)

In [12]:
image_encoder = CLIPVisionModel.from_pretrained(pretrained_image_encoder_path)

In [13]:
clip_image_processor = CLIPImageProcessor.from_pretrained(pretrained_image_encoder_path)

In [14]:
scheduler = DDPMScheduler.from_pretrained(unet_config_path, subfolder="scheduler")

The config attributes {'act_fn': 'silu', 'attention_head_dim': [5, 10, 20, 20], 'block_out_channels': [320, 640, 1280, 1280], 'center_input_sample': False, 'cross_attention_dim': 1024, 'down_block_types': ['CrossAttnDownBlock2D', 'CrossAttnDownBlock2D', 'CrossAttnDownBlock2D', 'DownBlock2D'], 'downsample_padding': 1, 'dual_cross_attention': False, 'flip_sin_to_cos': True, 'freq_shift': 0, 'in_channels': 4, 'layers_per_block': 2, 'mid_block_scale_factor': 1, 'norm_eps': 1e-05, 'norm_num_groups': 32, 'num_class_embeds': None, 'only_cross_attention': False, 'out_channels': 4, 'sample_size': 96, 'up_block_types': ['UpBlock2D', 'CrossAttnUpBlock2D', 'CrossAttnUpBlock2D', 'CrossAttnUpBlock2D'], 'use_linear_projection': True, 'upcast_attention': True} were passed to DDPMScheduler, but are not expected and will be ignored. Please verify your scheduler_config.json configuration file.


In [15]:
pipe = StableDiffusionReferenceOnlyPipeline(
    vae, image_encoder, clip_image_processor, unet, scheduler
)

/home/aihao/workspace/StableDiffusionReferenceOnly/src/stable_diffusion_reference_only/pipelines/stable_diffusion_reference_only_pipeline.py:94: FutureWarning: The configuration file of this scheduler: DDPMScheduler {
  "_class_name": "DDPMScheduler",
  "_diffusers_version": "0.21.0.dev0",
  "act_fn": "silu",
  "attention_head_dim": [
    5,
    10,
    20,
    20
  ],
  "beta_end": 0.02,
  "beta_schedule": "linear",
  "beta_start": 0.0001,
  "block_out_channels": [
    320,
    640,
    1280,
    1280
  ],
  "center_input_sample": false,
  "clip_sample": true,
  "clip_sample_range": 1.0,
  "cross_attention_dim": 1024,
  "down_block_types": [
    "CrossAttnDownBlock2D",
    "CrossAttnDownBlock2D",
    "CrossAttnDownBlock2D",
    "DownBlock2D"
  ],
  "downsample_padding": 1,
  "dual_cross_attention": false,
  "dynamic_thresholding_ratio": 0.995,
  "flip_sin_to_cos": true,
  "freq_shift": 0,
  "in_channels": 4,
  "layers_per_block": 2,
  "mid_block_scale_factor": 1,
  "norm_eps": 1e-05,


In [16]:
pipe.save_pretrained(
    "/home/aihao/workspace/DeepLearningContent/models/sd_reference_only/sd-2-1-init"
)